# 230614_pft_performance_comparison

# Import libraries

In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer
from datasets import load_dataset
from tqdm import tqdm
import json
import torch
import argparse
import datasets
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
from os import listdir
import json
import math
import seaborn as sns

/rds/general/user/jj1122/home/anaconda3/envs/m2d2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Config

In [2]:
sns.set_style("darkgrid")

In [3]:
# dataset_nm = "Health_and_fitness"
# dataset_path = f"../m2d2/dataset/{dataset_nm}/train.txt"
cache_dir="/rds/general/user/jj1122/ephemeral/.cache/huggingface"
data_dir = "../m2d2/dataset"

In [4]:
# def get_model_path_light(model_id):
#     model_path_format = "/rds/general/user/jj1122/home/projects/m2d2/dataset/{model_id}"
#     ckpt_path_format = "/checkpoint-{ckpt}"

#     if model_id == "gpt2":
#         model_path = "gpt2"
# #         ckpt = "zs"
#     else:
#         model_path = model_path_format.format(model_id=model_id)
#         l_dir = listdir(model_path)

#         if all([len(x.split(".")) == 1 for x in l_dir]):
#             ckpt = max([int(x.split("-")[1]) for x in l_dir])
#             model_path += ckpt_path_format.format(ckpt=ckpt)
# #         else:
# #             ckpt = "final"
#     return model_path

In [5]:
model_id_format = "pft_{gubun}_{threshold}_{ds}"
model_path_format = "/rds/general/user/jj1122/home/projects/m2d2/dataset/{model_id}"

In [19]:
list_gubuns = ["in", "all"]
list_thresholds = ["20", "40", "60", "80"]
# list_thresholds = ["60", "80"]

list_datasets = [
#     "cs_l1",
#     "nlin_l1",
#     "Health_and_fitness",
#     "Natural_and_physical_sciences",
#     "Religion_and_belief_systems",
#     "Culture_and_the_arts",
    "General_referece",
#     "econ_l1",
#     "History_and_events",
#     "Human_activites",
#     "Mathematics_and_logic",
#     "astro-ph_l1",
#     "cond-mat_l1",
#     "eess_l1",
#     "math_l1",
#     "physics_l1 (ERROR)",
#     "q-bio_l1",
#     "q-fin_l1",
#     "stat_l1",
#     "Philosophy",
#     "Philosophy_and_thinking",
#     "Society_and_social_sciences",
#     "Technology_and_applied_sciences",
]
list_model_ids = []
for gubun in list_gubuns:
    for threshold in list_thresholds:
        for ds in list_datasets:
            list_model_ids.append(model_id_format.format(gubun=gubun, threshold=threshold, ds=ds))
            
            
# list_model_ids = [model_path_format.format(model_id=model_id) for model_id in list_model_ids]

In [20]:
list_model_ids

['pft_in_20_General_referece',
 'pft_in_40_General_referece',
 'pft_in_60_General_referece',
 'pft_in_80_General_referece',
 'pft_all_20_General_referece',
 'pft_all_40_General_referece',
 'pft_all_60_General_referece',
 'pft_all_80_General_referece']

In [21]:
# device_id = 'cpu'
# n_layers = 12
# list_modules = ['attn', 'mlp']
# trace_module_id = "transformer.h.{l}.{m}"

In [22]:
# list_trace_module_ids = []

# for l in range(n_layers):
#     for m in list_modules:
#         list_trace_module_ids.append(trace_module_id.format(l=l, m=m))

# print(list_trace_module_ids[:3])

# Model

## load **17** finetuned models with EarlyStopper

In [23]:
config_models = {model_id: {} for model_id in list_model_ids}
config_models

{'pft_in_20_General_referece': {},
 'pft_in_40_General_referece': {},
 'pft_in_60_General_referece': {},
 'pft_in_80_General_referece': {},
 'pft_all_20_General_referece': {},
 'pft_all_40_General_referece': {},
 'pft_all_60_General_referece': {},
 'pft_all_80_General_referece': {}}

In [24]:
list_model_ids = list_model_ids + [list_datasets[0]]

In [25]:
config_models = {model_id: {} for model_id in list_model_ids}

for model_id in list_model_ids:
    if model_id.startswith("pft"):
        config_models[model_id]["model_path"] = model_path_format.format(model_id=model_id)
        json_path = config_models[model_id]["model_path"] + "/trainer_state.json"
        
    else:
        config_models[model_id]["model_path"] =model_path_format.format(model_id=model_id) + "/models"
        json_path = config_models[model_id]["model_path"] + "/trainer_state.json"
        
    with open(json_path, "r") as json_file:
        trainer_state = json.load(json_file)
        config_models[model_id]["perplexity"] = math.exp(trainer_state["best_metric"])
        config_models[model_id]["steps"] = int(trainer_state["best_model_checkpoint"].split("/")[-1].split("-")[-1])
        config_models[model_id]["eps"] = float(trainer_state["epoch"])

In [26]:
for ds, conf in config_models.items():
    print("{ds}: {ppl}".format(ds=ds, ppl=conf['perplexity']))

pft_in_20_General_referece: 23.059393324907077
pft_in_40_General_referece: 22.432933799968733
pft_in_60_General_referece: 22.151889596983786
pft_in_80_General_referece: 22.271078064725785
pft_all_20_General_referece: 23.096990325473765
pft_all_40_General_referece: 22.50587096542397
pft_all_60_General_referece: 22.198134399375448
pft_all_80_General_referece: 22.117281094327673
General_referece: 22.272872862552227


In [27]:
df_results = pd.DataFrame.from_dict(config_models).T.drop(["model_path"], axis=1).reset_index(names='model_id')

In [28]:
df_results["list_model_id"] = df_results["model_id"].apply(lambda model_id: model_id.split("_"))
df_results["gubun"] = df_results["list_model_id"].apply(lambda l: l[1] if l[0] == "pft" else "total")
df_results["threshold"] = df_results["list_model_id"].apply(lambda l: l[2] if l[0] == "pft" else "100").astype(int)
df_results["dataset"] = df_results["list_model_id"].apply(lambda l: "_".join(l[3:]) if l[0] == "pft" else "_".join(l))
del df_results["list_model_id"], df_results["model_id"]

In [29]:
df_results.loc[df_results["gubun"] != "total"].set_index(["dataset", "gubun", "threshold" ]).T

dataset    General_referece                                              
gubun                    in                                        all   
threshold                20         40         60         80        20   
perplexity        23.059393  22.432934   22.15189  22.271078  23.09699  \
steps                 31000      24000      25000      19000     34000   
eps               27.160494  21.399177  22.222222  17.283951  29.62963   

dataset                                      
gubun                                        
threshold          40         60         80  
perplexity  22.505871  22.198134  22.117281  
steps           24000      25000      25000  
eps         21.399177  22.222222  22.222222

In [30]:
df_results.loc[df_results["gubun"] == "total"].set_index(["dataset", "gubun", "threshold" ]).unstack(-1)

,,perplexity,steps,eps
,threshold,100,100,100
dataset,gubun,,,
General_referece,total,22.272873,19000,17.283951


In [9]:
base_model = GPT2LMHeadModel.from_pretrained("gpt2", cache_dir=cache_dir).to(device_id)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2", cache_dir=cache_dir)